In [12]:
import math
import random
import time
import pygame
pygame.init()

WIDTH, HEIGHT = 800, 600
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Aim Training")

TARGET_INCREMENT = 1500
TARGET_EVENT = pygame.USEREVENT

TARGET_PADDING = 30

BG_COLOUR = "light blue"
LIVES = 4
TOP_BAR_HEIGHT = 50

LABEL_FONT = pygame.font.SysFont("Helvetica", 24)
TITLE_FONT = pygame.font.SysFont("Helvetica", 40)

class Target:
    MAX_SIZE = 30
    GROWTH_RATE = 0.2
    COLOR = "red"
    SECOND_COLOR ="white"

    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.size = 0
        self.grow = True

    def update(self):
        if self.size + self.GROWTH_RATE >= self.MAX_SIZE:
            self.grow = False

        if self.grow:
            self.size += self.GROWTH_RATE
        else:
            self.size -= self.GROWTH_RATE

    def draw(self,win):
        pygame.draw.circle(win, self.COLOR, (self.x, self.y), self.size)
        pygame.draw.circle(win, self.SECOND_COLOR, (self.x, self.y), self.size* 0.8)
        pygame.draw.circle(win, self.COLOR, (self.x, self.y), self.size* 0.6)
        pygame.draw.circle(win, self.SECOND_COLOR, (self.x, self.y), self.size* 0.4)

    def collide(self, x, y):
        dis= math.sqrt((self.x - x)**2 + (self.y - y)**2)
        return dis <= self.size

def draw(win, targets):
    win.fill(BG_COLOUR)
    for target in targets:
        target.draw(win)

def format_time(secs):
    milli = math.floor(int(secs * 1000 % 1000) / 100)
    seconds = int(round(secs % 60, 1))
    minutes = int(secs // 60)
    return f"{minutes:02d}:{seconds:02d}.{milli}"

def draw_top_bar(win, elapsed_time, targets_pressed, misses):
    pygame.draw.rect(win, "white",(0, 0, WIDTH, TOP_BAR_HEIGHT))
    time_label = LABEL_FONT.render(f"Time: {format_time(elapsed_time)}", 1, "black")
    speed = round(targets_pressed / elapsed_time, 1) if elapsed_time > 0 else 0
    speed_label = LABEL_FONT.render(f"Speed: {speed} t/s", 1, "black")
    hits_label = LABEL_FONT.render(f"Hits: {targets_pressed}", 1, "black")
    lives_label = LABEL_FONT.render(f"Lives: {LIVES - misses}", 1, "black")
  
    win.blit(time_label, (5,5))
    win.blit(speed_label, (200,5))
    win.blit(hits_label, (450,5))
    win.blit(lives_label, (650,5))

def start_screen(win):
    win.fill(BG_COLOUR)
    
    title_label = TITLE_FONT.render("Aim Training", 1, "white")
    win.blit(title_label, (get_middle(title_label), HEIGHT // 2 - 100))

    instruction_label = LABEL_FONT.render("Click the targets as quickly as you can!", 1, "white")
    win.blit(instruction_label, (get_middle(instruction_label), HEIGHT // 2))

    start_label = LABEL_FONT.render("Press SPACE to start", 1, "white")
    win.blit(start_label, (get_middle(start_label), HEIGHT // 2 + 100))

    pygame.display.update()

    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    waiting = False
                elif event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    return False
    return True

def end_screen(win, elapsed_time, targets_pressed, clicks):
    win.fill(BG_COLOUR)
    
    title_label = TITLE_FONT.render("Game Over", 1, "white")
    win.blit(title_label, (get_middle(title_label), HEIGHT // 2 - 160))

    time_label = LABEL_FONT.render(f"Time: {format_time(elapsed_time)}", 1, "white")
    speed = round(targets_pressed / elapsed_time, 1) if elapsed_time > 0 else 0
    speed_label = LABEL_FONT.render(f"Speed: {speed} t/s", 1, "white")
    hits_label = LABEL_FONT.render(f"Hits: {targets_pressed}", 1, "white")
    accuracy = round(targets_pressed / clicks * 100, 1) if clicks > 0 else 0
    accuracy_label = LABEL_FONT.render(f"Accuracy: {accuracy}%", 1, "white")

    win.blit(time_label, (get_middle(time_label), HEIGHT // 2 - 80))
    win.blit(speed_label, (get_middle(speed_label), HEIGHT // 2 - 40))
    win.blit(hits_label, (get_middle(hits_label), HEIGHT // 2))
    win.blit(accuracy_label, (get_middle(accuracy_label), HEIGHT // 2 + 40))

    restart_label = LABEL_FONT.render("Press SPACE to restart or ESC to quit", 1, "white")
    win.blit(restart_label, (get_middle(restart_label), HEIGHT // 2 + 120))

    pygame.display.update()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    return True
                if event.key == pygame.K_ESCAPE:
                    return False
    return False

def get_middle(surface):
    return WIDTH // 2 - surface.get_width() // 2

def main():
    run = True
    while run:
        if not start_screen(WIN):
            run = False
            break

        targets = []
        clock = pygame.time.Clock()

        targets_pressed = 0
        clicks = 0
        misses = 0
        start_time = time.time()

        pygame.time.set_timer(TARGET_EVENT, TARGET_INCREMENT)
        
        game_active = True
        while game_active:
            clock.tick(60)
            click = False
            mouse_pos = pygame.mouse.get_pos()
            elapsed_time = time.time() - start_time
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return

                if event.type == TARGET_EVENT:
                    x = random.randint(TARGET_PADDING, WIDTH - TARGET_PADDING)
                    y = random.randint(TARGET_PADDING + TOP_BAR_HEIGHT, HEIGHT - TARGET_PADDING)
                    target = Target(x,y)
                    targets.append(target)

                if event.type == pygame.MOUSEBUTTONDOWN:
                    click = True
                    clicks += 1

            for target in targets[:]:
                target.update()

                if target.size <= 0:
                    targets.remove(target)
                    misses += 1

                if click and target.collide(*mouse_pos):
                    targets.remove(target)
                    targets_pressed += 1

            if misses >= LIVES:
                game_active = False
                
            draw(WIN, targets)
            draw_top_bar(WIN, elapsed_time, targets_pressed, misses)
            pygame.display.update()
        
        run = end_screen(WIN, elapsed_time, targets_pressed, clicks)

    pygame.quit()

if __name__ == "__main__":
    main()

In [14]:
import math
import random
import time
import pygame
pygame.init()

WIDTH, HEIGHT = 800, 600
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Aim Training")

TARGET_INCREMENT = 1500  # Initial spawn rate
TARGET_EVENT = pygame.USEREVENT

TARGET_PADDING = 30

BG_COLOUR = "light blue"
LIVES = 4
TOP_BAR_HEIGHT = 50

LABEL_FONT = pygame.font.SysFont("Helvetica", 24)
TITLE_FONT = pygame.font.SysFont("Helvetica", 40)

# New constant for speed increase
SPEED_INCREASE = 50  # Decrease spawn time by 50ms for each hit

class Target:
    MAX_SIZE = 30
    GROWTH_RATE = 0.2
    COLOR = "red"
    SECOND_COLOR ="white"

    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.size = 0
        self.grow = True

    def update(self):
        if self.size + self.GROWTH_RATE >= self.MAX_SIZE:
            self.grow = False

        if self.grow:
            self.size += self.GROWTH_RATE
        else:
            self.size -= self.GROWTH_RATE

    def draw(self,win):
        pygame.draw.circle(win, self.COLOR, (self.x, self.y), self.size)
        pygame.draw.circle(win, self.SECOND_COLOR, (self.x, self.y), self.size* 0.8)
        pygame.draw.circle(win, self.COLOR, (self.x, self.y), self.size* 0.6)
        pygame.draw.circle(win, self.SECOND_COLOR, (self.x, self.y), self.size* 0.4)

    def collide(self, x, y):
        dis= math.sqrt((self.x - x)**2 + (self.y - y)**2)
        return dis <= self.size

def draw(win, targets):
    win.fill(BG_COLOUR)
    for target in targets:
        target.draw(win)

def format_time(secs):
    milli = math.floor(int(secs * 1000 % 1000) / 100)
    seconds = int(round(secs % 60, 1))
    minutes = int(secs // 60)
    return f"{minutes:02d}:{seconds:02d}.{milli}"

def draw_top_bar(win, elapsed_time, targets_pressed, misses, current_speed):
    pygame.draw.rect(win, "white",(0, 0, WIDTH, TOP_BAR_HEIGHT))
    time_label = LABEL_FONT.render(f"Time: {format_time(elapsed_time)}", 1, "black")
    speed = round(targets_pressed / elapsed_time, 1) if elapsed_time > 0 else 0
    speed_label = LABEL_FONT.render(f"Speed: {speed} t/s", 1, "black")
    hits_label = LABEL_FONT.render(f"Hits: {targets_pressed}", 1, "black")
    lives_label = LABEL_FONT.render(f"Lives: {LIVES - misses}", 1, "black")
    current_speed_label = LABEL_FONT.render(f"Spawn Rate: {current_speed}ms", 1, "black")
  
    win.blit(time_label, (5,5))
    win.blit(speed_label, (200,5))
    win.blit(hits_label, (400,5))
    win.blit(lives_label, (550,5))
    win.blit(current_speed_label, (650,5))

def start_screen(win):
    win.fill(BG_COLOUR)
    
    title_label = TITLE_FONT.render("Aim Training", 1, "white")
    win.blit(title_label, (get_middle(title_label), HEIGHT // 2 - 100))

    instruction_label = LABEL_FONT.render("Click the targets as quickly as you can!", 1, "white")
    win.blit(instruction_label, (get_middle(instruction_label), HEIGHT // 2))

    start_label = LABEL_FONT.render("Press SPACE to start", 1, "white")
    win.blit(start_label, (get_middle(start_label), HEIGHT // 2 + 100))

    pygame.display.update()

    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    waiting = False
                elif event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    return False
    return True

def end_screen(win, elapsed_time, targets_pressed, clicks):
    win.fill(BG_COLOUR)
    
    title_label = TITLE_FONT.render("Game Over", 1, "white")
    win.blit(title_label, (get_middle(title_label), HEIGHT // 2 - 160))

    time_label = LABEL_FONT.render(f"Time: {format_time(elapsed_time)}", 1, "white")
    speed = round(targets_pressed / elapsed_time, 1) if elapsed_time > 0 else 0
    speed_label = LABEL_FONT.render(f"Speed: {speed} t/s", 1, "white")
    hits_label = LABEL_FONT.render(f"Hits: {targets_pressed}", 1, "white")
    accuracy = round(targets_pressed / clicks * 100, 1) if clicks > 0 else 0
    accuracy_label = LABEL_FONT.render(f"Accuracy: {accuracy}%", 1, "white")

    win.blit(time_label, (get_middle(time_label), HEIGHT // 2 - 80))
    win.blit(speed_label, (get_middle(speed_label), HEIGHT // 2 - 40))
    win.blit(hits_label, (get_middle(hits_label), HEIGHT // 2))
    win.blit(accuracy_label, (get_middle(accuracy_label), HEIGHT // 2 + 40))

    restart_label = LABEL_FONT.render("Press SPACE to restart or ESC to quit", 1, "white")
    win.blit(restart_label, (get_middle(restart_label), HEIGHT // 2 + 120))

    pygame.display.update()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    return True
                if event.key == pygame.K_ESCAPE:
                    return False
    return False

def get_middle(surface):
    return WIDTH // 2 - surface.get_width() // 2

def main():
    run = True
    while run:
        if not start_screen(WIN):
            run = False
            break

        targets = []
        clock = pygame.time.Clock()

        targets_pressed = 0
        clicks = 0
        misses = 0
        start_time = time.time()

        current_speed = TARGET_INCREMENT
        pygame.time.set_timer(TARGET_EVENT, current_speed)
        
        game_active = True
        while game_active:
            clock.tick(60)
            click = False
            mouse_pos = pygame.mouse.get_pos()
            elapsed_time = time.time() - start_time
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return

                if event.type == TARGET_EVENT:
                    x = random.randint(TARGET_PADDING, WIDTH - TARGET_PADDING)
                    y = random.randint(TARGET_PADDING + TOP_BAR_HEIGHT, HEIGHT - TARGET_PADDING)
                    target = Target(x,y)
                    targets.append(target)

                if event.type == pygame.MOUSEBUTTONDOWN:
                    click = True
                    clicks += 1

            for target in targets[:]:
                target.update()

                if target.size <= 0:
                    targets.remove(target)
                    misses += 1

                if click and target.collide(*mouse_pos):
                    targets.remove(target)
                    targets_pressed += 1
                    # Increase speed (decrease spawn time)
                    current_speed = max(200, current_speed - SPEED_INCREASE)
                    pygame.time.set_timer(TARGET_EVENT, current_speed)

            if misses >= LIVES:
                game_active = False
                
            draw(WIN, targets)
            draw_top_bar(WIN, elapsed_time, targets_pressed, misses, current_speed)
            pygame.display.update()
        
        run = end_screen(WIN, elapsed_time, targets_pressed, clicks)

    pygame.quit()

if __name__ == "__main__":
    main()